In [1]:
from coffea import hist
import math

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import mplhep as hep
import numpy as np
from itertools import chain

plt.style.use(hep.style.CMS)

import awkward as ak

from matplotlib import colors
POPTS={'norm':colors.LogNorm()}

In [2]:
from data import getData, repackage
ldmx_dict = getData(chunks=True, fnames="/Users/chloeg/Desktop/Work/Fermilab2021/HistData/kaon_pn_4GeV_Jul14_ntuple/*.root")

In [3]:
def extend_array(arr, new_attr, new_attr_name):
    members={n:arr[n] for n in arr.fields}
    members[new_attr_name] = new_attr
    return ak.zip(members)

def add_angle(arr,br=['px','py','pz','e'],name="theta"):
    from coffea.nanoevents.methods import vector
    ak.behavior.update(vector.behavior)

    part =  ak.zip({"x": arr.px,
                    "y": arr.py,
                    "z": arr.pz,
                    "t": arr.e,
                    },
                    with_name="LorentzVector")
    arr = extend_array(arr, part.theta, name)
    return arr

def get_vector(arr):
    from coffea.nanoevents.methods import vector
    ak.behavior.update(vector.behavior)

    part =  ak.zip({"x": arr.px,
                    "y": arr.py,
                    "z": arr.pz,
                    "t": arr.e,
                    },
                    with_name="LorentzVector")
    return part

def flat(x,axis=None): # for now must cast while waiting for coffea to catch up
    try:
        return ak.to_numpy(ak.flatten(x,axis=axis)) 
    except:
        return x

check if any kaon has 0 pn particles produced with it (n_otherparticles = ak.sum(PN_particles.pdgid != that kaon), if there is any case where n_otherparticles=0) : check that kaon energy

In [4]:
hists = {}

hists["hist_pdgid"] = hist.Hist("Sim Particle",
                                hist.Cat("Ptype", "Type of Particle"),
                                hist.Bin("e", r"PDG ID", 40, -400, 4000)
                            )
hists["hist_pdgid2"] = hist.Hist("Sim Particle",
                                hist.Cat("Ptype", "Type of Particle"),
                                hist.Bin("e", r"PDG ID", 40, -400, 4000)
                            )
hists["hist_ke"] = hist.Hist("Sim Particle",
                                hist.Cat("Ptype", "Type of Particle"),
                                hist.Bin("e", r"Kinetic Energy [MeV]", 50, 4, 3000),
                                )
hists["hist_ke2"] = hist.Hist("Sim Particle",
                                hist.Cat("Ptype", "Type of Particle"),
                                hist.Bin("e", r"Kinetic Energy [MeV]", 50, 4, 3000),
                                )

In [5]:
def ProcessChunk(chunk, hists):
    ldmx_events = repackage(chunk)
    
    sim_particle = ldmx_events['Sim_PNParticle']
    sim_particle_n = ldmx_events['n']['Sim_PNParticle']
    sim_kaon_n = ldmx_events['n']['Sim_Kaon']
    
    masks_id = {'Kplus': 321,
                'Kshort': 310,
                'Klong': 130, 
                'Kminus':-321,
                #'Electron': 11,
                #'Positron': -11,
                #'Pion': 211,
                #'Pi-': -211,
                #'Proton':2212,
                #'Neutron':2112,
                'Sigma': 3112 or 3212,
                'Lambda': 3122,
               }
    masses =   {'Kplus': 493.677,
                'Kshort': 493.677,
                'Klong': 493.677,
                'Kminus': 493.677,
                #'Electron': 0.511,
                #'Positron': 0.511,
                #'Pion': 139.57039,
                #'Pi-': 139.57039,
                #'Proton': 939.565,
                #'Neutron': 939.565,
                'Sigma': 1189.36,
                'Lambda': 1115.6,
                }
    
kaon_mask = sim_particle.pdgID == 321
    
    
#     index = ak.argsort(sim_particle.e, axis=-1, ascending=False, stable=True, highlevel=True, behavior=None)
#     index = ak.pad_none(index, 2, axis=1)

#     padded_pdg = ak.pad_none(sim_particle.pdgID, 2, axis=1)
#     pdgID_1 = padded_pdg[index][:,0]
#     pdgID_2 = padded_pdg[index][:,1]
#     filled_pdg1 = ak.fill_none(pdgID_1, -1000000)
#     filled_pdg2 = ak.fill_none(pdgID_2, -1000000)
    
#     padded_e = ak.pad_none(sim_particle.e, 2, axis=1)
#     e_1 = padded_e[index][:,0]
#     e_2 = padded_e[index][:,1]
#     filled_e1 = ak.fill_none(e_1, -1000000)
#     filled_e2 = ak.fill_none(e_2, -1000000)
    
    for element in sim_particle.pdgID:
        if 321 in element:
            if -321 not in element:
                if 130 not in element:
                    if 310 not in element:
                        for part,pmask in masks_id.items():
                            part_mask = (sim_particle.pdgID == pmask)
                            hists["hist_pdgid"].fill(Ptype= part,
                                     e=flat(sim_particle.pdgID[part_mask]),
                                     )  
                            #hists["hist_ke"].fill(Ptype= part,
                            #         e=flat(sim_particle.e[part_mask])- masses[part],
                            #         ) 
    

In [6]:
nchunk = 0
#total_particle = 0
#total_electron = 0
#total_sigma    = 0
#total_kaon     = 0
#total_pion     = 0
#total_proton   = 0
#total_neutron  = 0
#total_lambda   = 0

for chunk in ldmx_dict:
    nchunk += 1
    print('process',nchunk)
    ProcessChunk(chunk, hists)

    
#print('All Particles', total_particle, 
#'Electron', total_electron, 
#'Sigma', total_sigma,
#'Kaon', total_kaon,
#'Pion', total_pion,
#'Proton', total_proton,
#'Neutron', total_neutron,
#'Lambda', total_lambda)
    

process 1
process 2
process 3
process 4


KeyboardInterrupt: 

In [ ]:
#48 kminus
print(hists["hist_pdgid"])
fig, ax = plt.subplots(1,2, figsize=(24,10), constrained_layout=True)
hist.plot1d(hists["hist_pdgid"],ax=ax[0],clear=False);
#leg = ax[0].legend([r'$e^-$', r'$K^+$', r'$K_s$', r'$K_L$', r'$K^-$', r'$\Lambda$',r'$n^0$', r'$\pi^-$', r'$\pi^+$', r'$e^+$', r'$p^+$', r'$\Sigma$' ])
#leg = ax[0].legend([r'$K_L$', r'$K^-$', r'$K^+$', r'$K_s$', r'$\Lambda$', r'$\Sigma$'])
#leg = ax[0].legend([r'$K_L$', r'$K^-$', r'$K^+$', r'$K_s$'])


hist.plot1d(hists["hist_ke"],ax=ax[1],clear=False);
#leg = ax[1].legend([r'$e^-$', r'$K^+$', r'$K_s$', r'$K_L$', r'$K^-$', r'$\Lambda$',r'$n^0$', r'$\pi^-$', r'$\pi^+$', r'$e^+$', r'$p^+$', r'$\Sigma$' ])
#leg = ax[1].legend([r'$K_L$', r'$K^-$', r'$K^+$', r'$K_s$', r'$\Lambda$', r'$\Sigma$'])
#leg = ax[1].legend([r'$K_L$', r'$K^-$', r'$K^+$', r'$K_s$'])

